In [6]:
#Import libraries
import requests
from bs4 import BeautifulSoup
import json
import html
import html5lib
import os
import re

In [7]:
# Functions and data

# Define a dictionary with all the news options
news_sites = [
    {
        "name": "El pais",
        "url": "https://elpais.com/",
        "header": {
            "tag": "div",
            "class": "sm _df"
        },
        "classes": [
            {"tag": "h2", "class": "c_t", "title": "Titulares"},
            {"tag": "h2", "class": "c_t c_t-i", "title": "Opiniones"},
            {"tag": "div", "class": "b-t_a _df", "title": "Lo mas visto"}
        ],
        "particular_new": {
            "tag": "h1",
            "class": ["a_t"]
        }
    },
    {
        "name": "NBC",
        "url": "https://www.nbcnews.com/",
        "header": {
            "tag": "div",
            "class": "menu-section menu-section-sections menu-section-main"
        },
        "classes": [
            {"tag": "h2", "class": ["article-hero-headline__htag lh-none-print black-print article-hero-headline__htag--live-breaking", "multistoryline__headline founders-cond fw6 large noBottomSpace"], "title": "Header news"},
            {"tag": "h2", "class": "styles_headline__ice3t", "title": "Top stories and videos"}
        ],
        "particular_new": {
            "tag": "h1",
            "class": ["article-hero-headline__htag lh-none-print black-print"]
        }
    }
]

def get_user_web_selection(user_input_1):
    for new in news_sites:
        if new["name"].lower() == user_input_1.lower():
            return new
    print("El nombre del diario aportado no existe")
    return None

def scrape_web_header(url, header_tag, header_class):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml')
        header_div = soup.find(header_tag, class_=re.compile(r'\b' + re.escape(header_class) + r'\b'))
        if header_div:
            header_options = header_div.find_all("a")
            return [{"section": option.get_text(strip=True), "url": option.get("href"), "categories": []} for option in header_options]
        else:
            print("No se encontró el div con las opciones del encabezado.")
    else:
        print(f"Error al acceder a {url}")
    return []

def scrape_news(user_section_selection, user_web_selection):
    url = user_section_selection["url"]
    response = requests.get(url)
    response.encoding = 'utf-8'
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html5lib')
        categories = []
        for dicc in user_web_selection["classes"]:
            news_tags = soup.find_all(dicc["tag"], class_=dicc["class"])
            news_links = [link.a["href"] for link in news_tags]
            category = {"category_name": dicc["title"], "news": []}
            for news_url in news_links:
                news_response = requests.get(news_url)
                news_soup = BeautifulSoup(news_response.content, 'lxml')
                title = news_soup.find(user_web_selection["particular_new"]["tag"], class_=user_web_selection["particular_new"]["class"])
                paragraphs = news_soup.find_all('p')
                article_text = ' '.join([html.unescape(p.get_text(strip=True)) for p in paragraphs])
                if title:
                    category["news"].append({
                        "new_title": title.text,
                        "new": article_text
                    })
            categories.append(category)
        for option in web_sections:
            if option["section"] == user_section_selection["section"]:
                option["categories"] = categories
    else:
        print(f"Error al acceder a {url}")

def save_text_to_file(file_path, text):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)



In [8]:
# Main Execution
user_input_1 = input("Ingrese el nombre del periódico que desea resumir: ").strip()
user_web_selection = get_user_web_selection(user_input_1)

if user_web_selection:
    web_sections = scrape_web_header(user_web_selection["url"], user_web_selection["header"]["tag"], user_web_selection["header"]["class"])
    user_input_2 = input("Ingrese el nombre de la sección que desea buscar: ").strip()
    user_section_selection = next((item for item in web_sections if item["section"].lower() == user_input_2.lower()), None)
    
    if user_section_selection:
        scrape_news(user_section_selection, user_web_selection)
        web_scrapping_text = ""
        for item in web_sections:
            if item["categories"]:
                for category in item["categories"]:
                    web_scrapping_text += category["category_name"] + "\n"
                    for news_item in category["news"]:
                        web_scrapping_text += news_item["new_title"] + "\n"
                        web_scrapping_text += news_item["new"] + "\n"
                        web_scrapping_text += "\n"
        save_text_to_file(os.path.join('processed_data', 'web_scrapping_text.txt'), web_scrapping_text)
    else:
        print(f"No se encontró la sección '{user_input_2}' en el diccionario.")

In [9]:
print(web_scrapping_text)

Titulares
El líder de Hezbolá reta a Israel a invadir Líbano y promete un “justo castigo” al ataque con buscas y ‘walkie-talkies’ explosivos
Era el discurso más esperado del líder de Hezbolá en casi un año de guerra. Hasan Nasralá no tenía previsto hablar hasta dentro de tres semanas, con motivo del primer aniversario dela guerra en Gazay la apertura del “frente de apoyo” en Líbano, pero lo ha acabado haciendo este jueves, con tono grave y el mundo pendiente de sus palabras tras el mayor golpe al partido-milicia en sus cuatro décadas de historia:la doble detonación masiva a distancia de alrededor de 5.000 buscasywalkie-talkiesque había encargado hace meses. Tras la vulnerabilidad mostrada por el pirateo de sus métodos de comunicación, sobre cuya autoría (Israel) apenas hay dudas, Nasralá ha prometido una represalia “justa” y un “ajuste de cuentas grave” que solo se sabrá “cuando se vea”. “Permitidme cambiar de método: no hablaré del momento, ni del lugar de la respuesta. Por supuesto q